# Exercices Seance 06

## 1/ XML to CSV

Écrire une fonction qui lit les 5 fichiers XML de Balzac, créé et sauvegarde un dataframe avec 8 colonnes : titre, auteur, gender, genre, canon, date_publication, date_naissance, date_mort


In [1]:
import glob as glob
from lxml import etree
import pandas as pd 
import json
from collections import Counter

In [2]:
def XMl_to_df() :
    chemin_dossier='../16.03_Data/*Balzac*.xml'
    liste_xml=[i for i in glob.glob(chemin_dossier)]
    df=pd.DataFrame(columns=["titre", "auteur", "gender", "genre", "canon", "date_publication", "date_naissance", "date_mort"])
    for i,chemin_fichier in enumerate(liste_xml):
        arbre=etree.parse(chemin_fichier)
        titre=arbre.find('.//title').text
        auteur=arbre.find('.//author').get('name')
        gender=arbre.find('.//author').get('sex')
        genre=arbre.find('.//term').text
        canon = 'nani'
        date_pubication=arbre.find('//date').get('when')
        date_naissance, date_mort =arbre.find('//author').get('from') ,arbre.find('//author').get('to')
        ligne=[titre,auteur,gender,genre,canon,date_pubication,date_naissance,date_mort]
        df.loc[i]=ligne
    return df

In [6]:
XMl_to_df()

,titre,auteur,gender,genre,canon,date_publication,date_naissance,date_mort
0,Le Père Goriot,"Balzac, Honoré de",male,cycles et séries,nani,1843,1799,1850
1,Sarrasine,"Balzac, Honoré de",male,cycles et séries,nani,1844,1799,1850
2,Eugénie Grandet,"Balzac, Honoré de",male,cycles et séries,nani,1843,1799,1850
3,Le Chef-d'œuvre inconnu,"Balzac, Honoré de",male,cycles et séries,nani,1845,1799,1850
4,Illusions perdues,"Balzac, Honoré de",male,cycles et séries,nani,1843,1799,1850


## 2/ XML to txt 

Écrire une fonction qui convertie les 5 fichiers XML en fichiers txt.

In [1]:
def xml_to_txt():
#Boucle qui parcourt les 5 fichiers XML 
    chemin_dossier='../16.03_Data/*Balzac*.xml'
    liste_xml=[i for i in glob.glob(chemin_dossier)]
    for xml in liste_xml : 
#Lecture du fichier
        tree=etree.parse(xml)
#extraction et stock du texte
        contenu_texte = ''.join(tree.xpath('//text()'))
#Ecrire les str dans un ficher texte
        titre_txt=xml.split('/')[2].split('.')[0] +'.txt'
        with open(titre_txt,'w') as texte:
             texte.write(contenu_texte)
    return 
xml_to_txt()

NameError: name 'glob' is not defined

## 3/ Exploiter des JSON

À partir des .book (json customisés) envoyés avec ce notebook, écrire une fonction qui ouvre les documents, une autre qui récupère la liste des verbes ('agent', 'patient') et des adjectifs ('mod') associés à ces personnages (les 10 premiers), une autre qui calcule leur fréquence d'apparition relative au nombre total de verbes / adjectifs et retourne le résultat sour la forme d'un dataframe (colonnes = mots, ligne = txt_j_personnage_i, valeurs = fréquence relative).

Enfin définir une fonction main qui appelle ces différentes fonctions, fusionne les dataframes au fur et à mesure puis retourne le résultat sous la forme d'un dataframe (colonnes = mots, ligne = txt_j_personnage_i, valeurs = fréquence relative). 

Sauver ce dataframe final. Me l'envoyer avec votre notebook. 

Vous pouvez utiliser la fonction get_characterization vue dans le cours précédent.

Vous pouvez éggalement utiliser les fonctions déjà définies dans les différents exercices corrigés des semaines précédentes

In [2]:
def ouverture(chemin):
    with open(chemin, 'r') as fichier :
        data=json.load(fichier)
    return data,chemin.split('/')[1].split('.')[0]

In [16]:
def dict_prsg_vb_ajd(book,livre_nom,affichage:bool):
    list_verbs , list_adj = [], []
    dico_finale={}
    i=0
    while len(dico_finale.keys())<10 : 
            try: 
                personnage=book['characters'][i]['mentions']['proper'][0]['n']
                list_verbs.extend([item['w'].lower() for item in book["characters"][i]['agent']])
                list_verbs.extend([item['w'].lower() for item in book["characters"][i]['patient']])
                list_adj.extend([item['w'].lower() for item in book["characters"][i]['mod']])
                if f'{livre_nom}_{personnage}' not in dico_finale.keys() :
                    dico_finale[f'{livre_nom}_{personnage}']={'verbes':list_verbs ,'adjectifs':list_adj}
                    if affichage :
                        print(f'{livre_nom}_{personnage}')
                else : 
                    if affichage :
                        print(f'{livre_nom}_{personnage} déja dans le dico')
            except :
                if affichage :
                    print('Erreur')
                pass
            i+=1
    return dico_finale
#C'est le foutoir         

In [4]:
def calcul_frequence(liste:list) :
    #Accéder à la liste dans le dictionnaire (rappel format dico = {personnage_1 : {verbe : liste, adj:list}, personnage_2 :{verbe:liste, adj:list}}
    # créer un dictionnaire mot / fréquence 
    dictionnaire_mot_frequence =dict(Counter(sorted(liste)))
    #depuis le dictionnaire : créer les colonnes du df
    headers = [i for i in dictionnaire_mot_frequence.keys()]
    nombre_total = len(dictionnaire_mot_frequence.keys())
    #ajouter les valeurs relatives (frequence/nb de verbes (ie . len(verbes())))
    liste_mot_frequence_relative =[(i,dictionnaire_mot_frequence[i]/nombre_total) for i in dictionnaire_mot_frequence.keys()]
    #Dans un joli df
    df_valeur=pd.DataFrame(columns=headers)
    df_valeur.loc[0]=[i[1] for i in liste_mot_frequence_relative]
    return df_valeur

In [10]:
data, livre =ouverture('booknlp_output_book/1930_Colette_Sido.book')[0] , ouverture('booknlp_output_book/1930_Colette_Sido.book')[1]
livre#get_characterization('booknlp_output_book/1830_Stendhal_Le-Rouge-et-le-noir.book')
type(data['characters'][0]['agent'][0]['i'])

int

In [5]:
def main(chemin_dossier,affichage:bool): 
    #Ouverture du fichier
    liste_book=[i for i in glob.glob(chemin_dossier)]
    if affichage :
        print(liste_book)
    #Récupération des verbes et adjectifs en fonction des personnage
    DataFrame_final_verbe=pd.DataFrame()
    DataFrame_final_adj=pd.DataFrame()
    for dot_book in liste_book : 
        book , nom_livre = ouverture(dot_book)[0] , ouverture(dot_book)[1]
        dico_mot=dict_prsg_vb_ajd(book,nom_livre,affichage)
        #Rappel : format dictionnaire {personnage_1 : {verbe : liste, adj:list}, personnage_2 :{verbe:liste, adj:list}}
        #Les verbes 
        for personnage in dico_mot.keys() :
            df_=calcul_frequence(dico_mot[personnage]['verbes'])
            df_.insert(loc=0,column='Texte/personnage',value=personnage)
            DataFrame_final_verbe=pd.concat([DataFrame_final_verbe,df_])
            DataFrame_final_verbe.fillna(0,inplace=True)
            if affichage :
                print(DataFrame_final_verbe)
        #Les adjectifs
        for personnage in dico_mot.keys() :
            df_=calcul_frequence(dico_mot[personnage]['adjectifs'])
            df_.insert(loc=0,column='Texte/personnage',value=personnage)
            DataFrame_final_adj=pd.concat([DataFrame_final_adj,df_])
            DataFrame_final_adj.fillna(0,inplace=True)
            if affichage : 
                print(DataFrame_final_adj)
        
    # merge les df avec comme clé les noms des personnages
    results=pd.merge(DataFrame_final_verbe,DataFrame_final_adj,on='Texte/personnage')
    results.to_csv('DF_devoir.csv',index=False)
    return

In [6]:
main('booknlp_output_book/*.book',True)

['booknlp_output_book/1930_Colette_Sido.book', 'booknlp_output_book/1892_Verne-Jules_Le-Chateau-des-Carpathes.book', 'booknlp_output_book/1907_Leblanc-Maurice_Arsene-Lupin-gentleman-cambrioleur.book', 'booknlp_output_book/1830_Stendhal_Le-Rouge-et-le-noir.book', 'booknlp_output_book/1841_Sand-George_Un-hiver-a-Majorque.book', 'booknlp_output_book/1909_Zevaco-Michel_Nostradamus.book', 'booknlp_output_book/1883_Guy-de-Maupassant_Contes-de-la-Becasse.book']
Erreur
Erreur
Erreur
Erreur
Erreur
Erreur
Erreur
1930_Colette_Sido_« Sido
1930_Colette_Sido_Mme B
1930_Colette_Sido_Colette
1930_Colette_Sido_Antoine
Erreur
Erreur
Erreur
Erreur
Erreur
Erreur
Erreur
1930_Colette_Sido_Adrienne
1930_Colette_Sido_La pauvre Mme Enfert
1930_Colette_Sido_M. Rosimond
Erreur
Erreur
Erreur
Erreur
Erreur
Erreur
Erreur
Erreur
Erreur
1930_Colette_Sido_le pauvre M. Enfert
Erreur
1930_Colette_Sido_Mathieu
Erreur
Erreur
Erreur
Erreur
Erreur
Erreur
Erreur
Erreur
Erreur
Erreur
Erreur
Erreur
Erreur
Erreur
1930_Colette_S

In [15]:
def fonction_test():
    return 1,2
fonction_test()

(1, 2)